In [1]:
import datetime
import pandas as pd
from time import time
# from autograd import grad
# import autograd.numpy as np
import numpy as np
from numba import njit
from scipy.optimize import minimize, fsolve
import os

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score

In [3]:
train = pd.read_csv('/kaggle/input/Cascade_cup/train_age_dataset.csv')
test = pd.read_csv('/kaggle/input/Cascade_cup/test_age_dataset.csv')
sample = pd.read_csv('/kaggle/input/Cascade_cup/sample_submission.csv')

train = pd.get_dummies(train, columns=['tier','gender'])
test = pd.get_dummies(test, columns=['tier','gender'])

# REMEMBER TO ADD 1 IN THE END

train['age_group'] = train['age_group']-1

feature_cols = [col for col in train.columns.tolist() if col not in ['age_group']]
target_cols = ['age_group']

In [4]:
# Combine oof
oof_paths = ['../input/double-xgboost/oof_xgb.npy',
             '../input/xgboost-pca/oof_xgb.npy',
             '../input/xgboost-withunnamed-deb/oof_xgb_8066_8100.npy',
             '../input/xgboost-withunnamed-nonzerotraining/oof_xgb.npy',
             '../input/double-xgboost-manual-tuning/oof_xgb.npy',
             '../input/debrf/oof_rnd.npy',
             '../input/xgboost-withunnamed/oof_xgb.npy',
             '../input/catboost-tuned/oof_cat.npy',
             '../input/lightgbm/oof_lgb.npy', 
             '../input/deb-xgb/oof_xgb_75143.npy',
             '../input/randomforest/oof_rf.npy']

pred_paths = ['../input/double-xgboost/pred_xgb.npy',
              '../input/xgboost-pca/pred_xgb.npy',
              '../input/xgboost-withunnamed-deb/pred_xgb_8066_8100.npy',
              '../input/xgboost-withunnamed-nonzerotraining/pred_xgb.npy',
              '../input/double-xgboost-manual-tuning/pred_xgb.npy',
              '../input/debrf/pred_rnd.npy',
              '../input/xgboost-withunnamed/pred_xgb.npy',
              '../input/catboost-tuned/pred_cat.npy', 
              '../input/lightgbm/pred_lgb.npy', 
              '../input/deb-xgb/pred_xgb_75143.npy',
              '../input/randomforest/pred_rf.npy']


oof_comp = np.zeros((len(oof_paths), len(train[target_cols]), 4))
pred_comp = np.zeros((len(pred_paths), len(test), 4))

for i in range(len(oof_paths)):
    oof_comp[i, :, :] = np.load(oof_paths[i])
    
for i in range(len(pred_paths)):
    pred_comp[i, :, :] = np.load(pred_paths[i])

In [5]:
import tensorflow as tf
y_true = (train[target_cols].values).reshape(-1,1)
y_true_hot = np.array(tf.one_hot(y_true, depth=4)).reshape(-1,4)


def f1score(oof):
    predictions = oof.argmax(axis=1)
    return f1_score(train[target_cols], predictions, average='weighted')

# return negative of f1 to minimize
def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof_comp, axes=(0,0))
    return -f1score(oof_blend)



In [6]:
%%time

f1_scores = {}

for i in range(len(oof_paths)):
    score_oof = f1score(oof_comp[i, ...])
    f1_scores[oof_paths[i]] = score_oof
    print(f'{oof_paths[i]} CV:\t\t\t',score_oof)
print('-' * 50)

../input/double-xgboost/oof_xgb.npy CV:			 0.7899487480513191
../input/xgboost-pca/oof_xgb.npy CV:			 0.7890089004070117
../input/xgboost-withunnamed-deb/oof_xgb_8066_8100.npy CV:			 0.8066411146358526
../input/xgboost-withunnamed-nonzerotraining/oof_xgb.npy CV:			 0.8063223170728732
../input/double-xgboost-manual-tuning/oof_xgb.npy CV:			 0.74824771829688
../input/debrf/oof_rnd.npy CV:			 0.7661930512519203
../input/xgboost-withunnamed/oof_xgb.npy CV:			 0.8063599626965134
../input/catboost-tuned/oof_cat.npy CV:			 0.7651350154826301
../input/lightgbm/oof_lgb.npy CV:			 0.7697550527301407
../input/deb-xgb/oof_xgb_75143.npy CV:			 0.7514380843312148
../input/randomforest/oof_rf.npy CV:			 0.7630414696367479
--------------------------------------------------
CPU times: user 2.29 s, sys: 0 ns, total: 2.29 s
Wall time: 2.27 s


In [7]:
import scipy

tol = 1e-3
bnds = [(0, 1) for _ in range(oof_comp.shape[0])]
init_guess = [1 / oof_comp.shape[0]] * oof_comp.shape[0]
print('Inital Blend OOF:', func_numpy_metric(init_guess))
start_time = time()

res_scipy = scipy.optimize.differential_evolution(func = func_numpy_metric,
                                                 bounds=bnds,
                                                 disp=True, 
                                                 polish=True,
                                                 tol=tol)

print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] Optimised Blend OOF:', res_scipy.fun)
print('Optimised Weights:', res_scipy.x)

weights = res_scipy.x

Inital Blend OOF: -0.8031900561457075
differential_evolution step 1: f(x)= -0.809266
differential_evolution step 2: f(x)= -0.809841
differential_evolution step 3: f(x)= -0.810563
differential_evolution step 4: f(x)= -0.810563
differential_evolution step 5: f(x)= -0.810715
differential_evolution step 6: f(x)= -0.810715
differential_evolution step 7: f(x)= -0.810866
[04:54] Optimised Blend OOF: -0.8108656269005935
Optimised Weights: [0.22187325 0.18063738 0.97419533 0.75991482 0.02599876 0.10349734
 0.880284   0.06250516 0.05635466 0.50080978 0.12888972]


## CV HISTORY - 

OOF - 75.59    
LB - 75.27

######################

OOF - 0.75613   
LB - 75.308

######################

OOF - 0.7571 \
LB - 0.7541

####################

OOF - 0.7579 \
LB - 0.7549

In [8]:
def blend_preds(weights):
    for i in range(pred_comp.shape[0]):
        pred_blend = np.tensordot(weights, pred_comp, axes=(0,0))
        
    return pred_blend

pred_blend = blend_preds(weights)

In [9]:
final_preds = pred_blend.argmax(axis=1)+1

In [10]:
pred_csv = pd.DataFrame(final_preds.reshape(-1), columns=['prediction'] )
pred_csv

,prediction
0,1
1,1
2,1
3,2
4,1
...,...
54315,1
54316,1
54317,4
54318,1


In [11]:
pred_csv.value_counts()

prediction
1             33942
2              8597
3              8245
4              3536
dtype: int64

In [12]:
pred_csv.to_csv('pcawithdeb.csv', index=False)